<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#数据介绍" data-toc-modified-id="数据介绍-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>数据介绍</a></span></li><li><span><a href="#数据分析目的" data-toc-modified-id="数据分析目的-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>数据分析目的</a></span></li><li><span><a href="#描述性分析" data-toc-modified-id="描述性分析-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>描述性分析</a></span></li><li><span><a href="#基于SVD的模型" data-toc-modified-id="基于SVD的模型-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>基于SVD的模型</a></span><ul class="toc-item"><li><span><a href="#模型介绍" data-toc-modified-id="模型介绍-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>模型介绍</a></span></li><li><span><a href="#算法实现" data-toc-modified-id="算法实现-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>算法实现</a></span></li><li><span><a href="#训练模型" data-toc-modified-id="训练模型-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>训练模型</a></span></li><li><span><a href="#模型结论" data-toc-modified-id="模型结论-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>模型结论</a></span></li></ul></li><li><span><a href="#基于SVD++的模型" data-toc-modified-id="基于SVD++的模型-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>基于SVD++的模型</a></span><ul class="toc-item"><li><span><a href="#模型介绍" data-toc-modified-id="模型介绍-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>模型介绍</a></span></li><li><span><a href="#算法实现" data-toc-modified-id="算法实现-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>算法实现</a></span></li><li><span><a href="#训练模型" data-toc-modified-id="训练模型-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>训练模型</a></span></li><li><span><a href="#模型结论" data-toc-modified-id="模型结论-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>模型结论</a></span></li></ul></li><li><span><a href="#模型比较" data-toc-modified-id="模型比较-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>模型比较</a></span></li></ul></div>

## 数据介绍

** 这个数据集包含以下信息：**

* 来源：美国明尼苏达州立大学的GroupLens的研究项目
* 日期：1997/9/10至1988/4/22
* 来自943用户关于1682部电影的100000 评分 (1-5)  
* 每个用户最少会评论20部电影


## 数据分析目的

** 这个数据集实际上不具有任何商业价值，因为是30年前的数据，缺乏推荐的时效性。本文目的是为了实现两种基于SVD的推荐系统的算法. **

## 描述性分析

In [17]:
import numpy as np  
import random  
import pandas as pd 
def getData():   #获取训练集和测试集的函数  
    data=pd.read_table('/Users/mybook/Desktop/电影推荐系统/ml-100k/u.data',names=['userId','movieId','rating','date'])
    data=np.array(data.iloc[:,:3]).tolist()
    np.random.seed(1234)
    random.shuffle(data) 
    train_data=data[:int(len(data)*0.8)]
    test_data=data[int(len(data)*0.8):] 
    print('load data finished')  
    print('total data ',len(data))    
    return train_data,test_data,data
train_data,test_data,data=getData()  
df=pd.DataFrame(data)
df.iloc[:,2].describe()

load data finished
total data  100000


count    100000.000000
mean          3.529860
std           1.125674
min           1.000000
25%           3.000000
50%           4.000000
75%           4.000000
max           5.000000
Name: 2, dtype: float64

* 数据data包含了10000组数据：的第一列描述的是用户的id信息，第二列描述的是电影的id的信息，id只是用数字代替用户和电影，不包含其他信息。第三列是所有电影评分。

* 通过简单的统计，可以看出评分总体是呈左偏分布的。

## 基于SVD的模型

### 模型介绍

$$R=P_{U\times K}Q_{K\times I}$$

$$\hat{r}_{ui}=p^T_\mu q_i$$

$$\hat{r_{ui}}=\mu+b_\mu+b_i+p^T_\mu q_i$$

* $每个评分矩阵R都可以由SVD分解成矩阵$P$和$Q$相乘的形式，其中每个元素$\hat{r}_{ui}$代表用户u对电影i的评分，总可以写成$P$的第u行和$Q$第i列乘积表示，只要估计出$P$和$Q$，就可以给用户未评分的电影估一个评分。

* 可以由已知的评分$r$去训练出$P$和$Q$，误差为$$e_{ui}=r-\hat{r}_{ui}$$,获得是误差平方和SSE最小的$P$和$Q$

* 因为考虑每个用户有不同的行为特征，在原来的模型中加入一项$b_\mu$,同理考虑每部电影的异质性，再加入$b_i$,还应加入平均评分估计量$\mu$

* 因为要估计的参数很多，为了防止过拟合，将模型改成加惩罚的优化模型,根据模型的含义结合最适合的罚函数Ridge，即RSVD模型：

$$SSE=\sum_{ui}{e_{ui}}^{2}+\lambda \sum_{u}{b_\mu}^{2}+\lambda\sum_{u}{b_i}^{2}+\lambda\sum_{u}\{{\left|p_\mu\right|}^{2}+\lambda\sum_{i}{\left|q_i\right|}^{2}$$

*优化模型结合选择随机梯度下降法更新迭代，具体算法如下：*

### 算法实现

In [23]:
%%time
import time
class SVD:  
    def __init__(self,mat,K=20):  
        self.mat=np.array(mat)  
        self.K=K  
        self.bi={}  
        self.bu={}  
        self.qi={}  
        self.pu={}  
        self.avg=np.mean(self.mat[:,2])  
        for i in range(self.mat.shape[0]):  
            uid=self.mat[i,0]  
            iid=self.mat[i,1]  
            self.bi.setdefault(iid,0)  
            self.bu.setdefault(uid,0)  
            self.qi.setdefault(iid,np.random.random((self.K,1))/10*np.sqrt(self.K))  
            self.pu.setdefault(uid,np.random.random((self.K,1))/10*np.sqrt(self.K)) 
            
    def predict(self,uid,iid):  #预测评分的函数  
        #setdefault的作用是当该用户或者物品未出现过时，新建它的bi,bu,qi,pu，并设置初始值为0  
        self.bi.setdefault(iid,0)  
        self.bu.setdefault(uid,0)  
        self.qi.setdefault(iid,np.zeros((self.K,1)))  
        self.pu.setdefault(uid,np.zeros((self.K,1)))  
        rating=self.avg+self.bi[iid]+self.bu[uid]+np.sum(self.qi[iid]*self.pu[uid]) #预测评分公式  
        #由于评分范围在1到5，所以当分数大于5或小于1时，返回5,1.  
        if rating>5:  
            rating=5  
        if rating<1:  
            rating=1  
        return rating
    
      
    def train(self,steps=30,gamma=0.04,Lambda=0.15):    #训练函数，step为迭代次数。  
        print('train data size',self.mat.shape)  
        for step in range(steps):  
            print('step',step+1,'is running')  
            KK=np.random.permutation(self.mat.shape[0]) #随机梯度下降算法，kk为对矩阵进行随机洗牌  
            rmse=0.0  
            for i in range(self.mat.shape[0]):  
                j=KK[i]  
                uid=self.mat[j,0]  
                iid=self.mat[j,1]  
                rating=self.mat[j,2]  
                eui=rating-self.predict(uid, iid)  
                rmse+=eui**2  
                self.bu[uid]+=gamma*(eui-Lambda*self.bu[uid])    
                self.bi[iid]+=gamma*(eui-Lambda*self.bi[iid])  
                tmp=self.qi[iid]  
                self.qi[iid]+=gamma*(eui*self.pu[uid]-Lambda*self.qi[iid])  
                self.pu[uid]+=gamma*(eui*tmp-Lambda*self.pu[uid])  
            gamma=0.93*gamma  
            print('rmse is',np.sqrt(rmse/self.mat.shape[0]))  
      
    def test(self,test_data):  #gamma以0.93的学习率递减  
        test_data=np.array(test_data)  
        print('test data size',test_data.shape)  
        rmse=0.0  
        for i in range(test_data.shape[0]):  
            uid=test_data[i,0]  
            iid=test_data[i,1]  
            rating=test_data[i,2]  
            eui=rating-self.predict(uid, iid)  
            rmse+=eui**2  
        print('rmse of test data is',np.sqrt(rmse/test_data.shape[0]))
            
a=SVD(train_data,30)    
a.train()  
a.test(test_data)  



train data size (80000, 3)
step 1 is running
rmse is 1.06734820787
step 2 is running
rmse is 0.936166287804
step 3 is running
rmse is 0.920096142613
step 4 is running
rmse is 0.90967978745
step 5 is running
rmse is 0.901208234491
step 6 is running
rmse is 0.892747388361
step 7 is running
rmse is 0.884952249174
step 8 is running
rmse is 0.878056584269
step 9 is running
rmse is 0.872322403522
step 10 is running
rmse is 0.867550227558
step 11 is running
rmse is 0.862861017231
step 12 is running
rmse is 0.858990351551
step 13 is running
rmse is 0.855066866182
step 14 is running
rmse is 0.852392967734
step 15 is running
rmse is 0.849529584335
step 16 is running
rmse is 0.84698677619
step 17 is running
rmse is 0.844568531023
step 18 is running
rmse is 0.842629636075
step 19 is running
rmse is 0.840624112574
step 20 is running
rmse is 0.838878048016
step 21 is running
rmse is 0.837005283325
step 22 is running
rmse is 0.83575339439
step 23 is running
rmse is 0.834341790882
step 24 is running
r

### 训练模型

** 调整参数，训练集的最小rmse为0.8271，测试集最小结果为0.9126,算法运行时间为1分33秒 **


*下面用训练出来的模型对缺失进行预测:*

In [24]:
#对缺值进行评分
bi=a.bi  
bu=a.bu
qi=a.qi       
pu=a.pu
avg=a.avg
data=np.array(data)
rate={}
for i in set(data[:,0]):
    d=data[data[:,0]==i,]
    for j in set(data[:,1]):
        if np.count_nonzero(d[d[:,1]==j,][:,2])==0: 
            rate[(i,j)]=avg+bi[j]+bu[i]+np.sum(qi[j]*pu[i])
          
#将评分高于4分的电影推荐给用户
l={}
for i in rate.keys():
    if rate[i]>=4:
        l[i]=rate[i]
l

{(1, 275): 4.0343913006775693,
 (1, 276): 4.0714592428767649,
 (1, 285): 4.3577983544815027,
 (1, 302): 4.3532053666963355,
 (1, 306): 4.0846457599098347,
 (1, 313): 4.0899605456003183,
 (1, 315): 4.1608960837573141,
 (1, 316): 4.1132060079844273,
 (1, 318): 4.3544748479004456,
 (1, 320): 4.1365295284325434,
 (1, 357): 4.3034328906871853,
 (1, 408): 4.6099067085787278,
 (1, 425): 4.0253781565365463,
 (1, 427): 4.2935615330662262,
 (1, 428): 4.1330588887949284,
 (1, 429): 4.0059940268088416,
 (1, 430): 4.1796469672983676,
 (1, 474): 4.4169770773126276,
 (1, 475): 4.0470643833975206,
 (1, 478): 4.1420297448331844,
 (1, 479): 4.3027674571966879,
 (1, 480): 4.3615061451056194,
 (1, 482): 4.1076490271559045,
 (1, 483): 4.4838882210865467,
 (1, 484): 4.2383472717546677,
 (1, 487): 4.1160440042909423,
 (1, 488): 4.2337711777585483,
 (1, 489): 4.1546320534822092,
 (1, 493): 4.1760970246068849,
 (1, 494): 4.1740532380167572,
 (1, 496): 4.0452700450081327,
 (1, 497): 4.0225262326916056,
 (1, 498

In [8]:
print('total rating data',len(rate))
print('total rating data larger than 4',len(l))

total rating data 1486126
total rating data 244073


### 模型结论

**通过模型进行预测，获得1486126个评分，其中大于4分的有244073个，对这些用户推荐相应电影**

## 基于SVD++的模型
 
### 模型介绍

* 在SVD模型基础上引入隐式反馈，用户对电影的历史评分数据等作为新的参数。

* 现实中，隐含回馈的原因比较复杂，可以使用用户的历史浏览数据、用户历史评分数据、电影的历史浏览数据、电影的历史评分数据等作为新的参数。隐含的反馈可以是打分动作（谁对某个商品打过分），或者是浏览记录等。只要有类似的隐含反馈，客观上也表示了user对某item的偏好，应该专门给一部分参数空间去建模。

* SVD++模型为： 

$$\hat{r}_{ui}=\mu+b_\mu+b_i+q^T_i(p_\mu+\frac{\sum_{{j}\in{I_u}}{y_j}}{\sqrt{N(I_u)}})$$


** 它的含义是这样的：评分行为从侧面反映了用户的喜好，可以将这样的反映通过隐式参数形式体系在模型中，就是上式相比于（3）式的增加的部分，其中$I_u$是用户u评论过的物品的集合，每部item对应一个向量 $y_i$，通过user隐含回馈过的item的集合来刻画用户的偏好。也通过梯度下降算法优化,这里的-1/2是个经验值。**




### 算法实现

In [13]:
class SVDPP:  
    def __init__(self,mat,K=20):  
        self.mat=np.array(mat)  
        self.K=K  
        self.bi={}  
        self.bu={}  
        self.qi={}  
        self.pu={}  
        self.avg=np.mean(self.mat[:,2])  
        self.y={}  
        self.u_dict={}  
        for i in range(self.mat.shape[0]):  
              
            uid=self.mat[i,0]  
            iid=self.mat[i,1]  
            self.u_dict.setdefault(uid,[])  
            self.u_dict[uid].append(iid)  
            self.bi.setdefault(iid,0)  
            self.bu.setdefault(uid,0)  
            self.qi.setdefault(iid,np.random.random((self.K,1))/10*np.sqrt(self.K))  
            self.pu.setdefault(uid,np.random.random((self.K,1))/10*np.sqrt(self.K))  
            self.y.setdefault(iid,np.zeros((self.K,1))+.1)  
    def predict(self,uid,iid):  #预测评分的函数  
        #setdefault的作用是当该用户或者物品未出现过时，新建它的bi,bu,qi,pu及用户评价过的物品u_dict，并设置初始值为0  
        self.bi.setdefault(iid,0)  
        self.bu.setdefault(uid,0)  
        self.qi.setdefault(iid,np.zeros((self.K,1)))  
        self.pu.setdefault(uid,np.zeros((self.K,1)))  
        self.y.setdefault(uid,np.zeros((self.K,1)))  
        self.u_dict.setdefault(uid,[])  
        u_impl_prf,sqrt_Nu=self.getY(uid, iid)  
        rating=self.avg+self.bi[iid]+self.bu[uid]+np.sum(self.qi[iid]*(self.pu[uid]+u_impl_prf)) #预测评分公式  
        #由于评分范围在1到5，所以当分数大于5或小于1时，返回5,1.  
        if rating>5:  
            rating=5  
        if rating<1:  
            rating=1  
        return rating  
      
    #计算sqrt_Nu和∑yj  
    def getY(self,uid,iid):  
        Nu=self.u_dict[uid]  
        I_Nu=len(Nu)  
        sqrt_Nu=np.sqrt(I_Nu)  
        y_u=np.zeros((self.K,1))  
        if I_Nu==0:  
            u_impl_prf=y_u  
        else:  
            for i in Nu:  
                y_u+=self.y[i]  
            u_impl_prf = y_u / sqrt_Nu  
          
        return u_impl_prf,sqrt_Nu  
      
    def train(self,steps=30,gamma=0.04,Lambda=0.15):    #训练函数，step为迭代次数。  
        print('train data size',self.mat.shape)  
        for step in range(steps):  
            print('step',step+1,'is running')  
            KK=np.random.permutation(self.mat.shape[0]) #随机梯度下降算法，kk为对矩阵进行随机洗牌  
            rmse=0.0  
            for i in range(self.mat.shape[0]):  
                j=KK[i]  
                uid=self.mat[j,0]  
                iid=self.mat[j,1]  
                rating=self.mat[j,2]  
                predict=self.predict(uid, iid)  
                u_impl_prf,sqrt_Nu=self.getY(uid, iid)  
                eui=rating-predict  
                rmse+=eui**2  
                self.bu[uid]+=gamma*(eui-Lambda*self.bu[uid])    
                self.bi[iid]+=gamma*(eui-Lambda*self.bi[iid])  
                self.pu[uid]+=gamma*(eui*self.qi[iid]-Lambda*self.pu[uid])  
                self.qi[iid]+=gamma*(eui*(self.pu[uid]+u_impl_prf)-Lambda*self.qi[iid])  
                for j in self.u_dict[uid]:  
                    self.y[j]+=gamma*(eui*self.qi[j]/sqrt_Nu-Lambda*self.y[j])  
                                      
            gamma=0.93*gamma  
            print('rmse is',np.sqrt(rmse/self.mat.shape[0]))  
      
    def test(self,test_data):  #gamma以0.93的学习率递减  
          
        test_data=np.array(test_data)  
        print('test data size',test_data.shape)  
        rmse=0.0  
        for i in range(test_data.shape[0]):  
            uid=test_data[i,0]  
            iid=test_data[i,1]  
            rating=test_data[i,2]  
            eui=rating-self.predict(uid, iid)  
            rmse+=eui**2  
        print('rmse of test data is',np.sqrt(rmse/test_data.shape[0]))  
      
      
train_data,test_data,data=getData()  
a=SVDPP(train_data,30)    
a.train()  
a.test(test_data)  


load data finished
total data  100000
train data size (80000, 3)
step 1 is running
rmse is 1.04068452051
step 2 is running
rmse is 0.939950306621
step 3 is running
rmse is 0.922808264233
step 4 is running
rmse is 0.912891413725
step 5 is running
rmse is 0.904342353724
step 6 is running
rmse is 0.895858587282
step 7 is running
rmse is 0.888359997121
step 8 is running
rmse is 0.881292807956
step 9 is running
rmse is 0.875407714112
step 10 is running
rmse is 0.869912780091
step 11 is running
rmse is 0.865661648134
step 12 is running
rmse is 0.861807637705
step 13 is running
rmse is 0.857985436683
step 14 is running
rmse is 0.854839518548
step 15 is running
rmse is 0.851757659014
step 16 is running
rmse is 0.848926864317
step 17 is running
rmse is 0.846945576994
step 18 is running
rmse is 0.844485274473
step 19 is running
rmse is 0.842783327412
step 20 is running
rmse is 0.840676314158
step 21 is running
rmse is 0.839057557173
step 22 is running
rmse is 0.837585918553
step 23 is running
rm

### 训练模型

** SVD++模型的精度比较SVD的0.9126稍有提高，rmse为0.9105，但是运行时间超过20分钟，时间成本太高，因此不再用这个模型对用户进行评分。**


### 模型结论

SVD++算法更适合对精度有超高要求的应用场景，这也是这种模型算法在各种算法比赛中大放异彩，但是在工业界却不太受青睐的原因，当然可以基于分布式或其他的方法进行改进，提高效率，例如 spark SVD++等。

## 模型比较

虽然SVD++的模型相比较简单SVD显得更加高级，但是，进行数据分析的目的并不只是看模型是不是更厉害，算法的预测效果是不是更好，考虑到模型与算法的简洁性与时效性，我认为SVD是一种更好用的算法，就像是朴素的logistic回归模型。